# Exercise 3 - Data Lake on S3

In [1]:
from pyspark.sql import SparkSession
import os
import configparser

# Make sure that your AWS credentials are loaded as env vars

In [2]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read_file(open('~/.aws/credentials'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

FileNotFoundError: [Errno 2] No such file or directory: '~/.aws/credentials'

# Create spark session with hadoop-aws package

In [ ]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Load data from S3

In [ ]:
df = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv")

In [ ]:
df.printSchema()
df.show(5)

# Infer schema, fix header and separator

In [ ]:
df = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv",sep=";", inferSchema=True, header=True)

In [ ]:
df.printSchema()
df.show(5)

# Fix the data yourself 

In [ ]:
import  pyspark.sql.functions as F
dfPayment = df.withColumn("payment_date", F.to_timestamp("payment_date"))
dfPayment.printSchema()
dfPayment.show(5)

# Extract the month

In [ ]:
dfPayment = df.withColumn("month", F.month("payment_date"))
dfPayment.show(5)

# Compute aggregate revenue per month

In [ ]:
dfPayment.createOrReplaceTempView("payment")
spark.sql(
"""
SELECT month, sum(amount) as revenue
FROM payment
GROUP BY month
ORDER BY revenue desc
"""
).show(5)

# Fix the schema

In [ ]:
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
paymentSchema = R([
    Fld("payment_id", Int()),
    Fld("customer", Int()),
    Fld("staff", Int()),
    Fld("rental", Int()),
    Fld("amount", Dbl()),
    Fld("payment_Date", Date())
])

In [ ]:
dfPaymentWithSchema = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv", sep=";", schema=paymentSchema, header=True)

In [ ]:
dfPaymentWithSchema.printSchema()
df.show(5)

In [ ]:
dfPaymentWithSchema.createOrReplaceTempView("payment")
spark.sql(
"""
SELECT month(payment_date) as m, sum(amount) as revenue
FROM payment
GROUP BY m
ORDER BY revenue desc
"""
).show()